In [3]:
import numpy as np
import pandas as pd
import itertools as iter
import warnings
import pickle

warnings.filterwarnings("ignore")
def new_dataframe():
    colNames = []
    for i in range(1, 13):
        colNames.append(f"coeff{i}")

    # colNames.append("speaker_count")
    # colNames.append("utter_count")
    # colNames.append("frame_count")

    index = pd.MultiIndex.from_tuples([], names=["speak", "utter", "frame"])
    df = pd.DataFrame(columns=colNames)
    print("Created Dataframe")
    return colNames, df

colNames, df = new_dataframe()

Created Dataframe


In [4]:
bounds = [i for i in iter.accumulate([30, 30, 30, 30, 30, 30, 30, 30, 30])]
print(bounds)

speak_count = 0
utter_count = 0
frame_count = -1

train_file = open("./data/ae.train", "r")
print("Loaded data")

for line in train_file.readlines():
    if line == "\n":
        utter_count +=1
        frame_count = -1
        speak_count = (utter_count//30)

    else:
        frame_count += 1

        row = np.asarray(line[:-2].split(" ")).reshape(1, 12)
        # row.append(speak_count)
        # row.append(utter_count)
        # row.append(frame_count)

        #print(row)

        utter_idx = utter_count - bounds[speak_count - 1] if speak_count > 0 else utter_count
        # print(utter_idx)
        index = pd.MultiIndex.from_tuples([(int(speak_count), int(utter_idx), int(frame_count))], names=["speak", "utter", "frame"])
        row = pd.DataFrame(row, index=index, columns=colNames)

        #print(row)
        df = pd.concat([df, row])

print("Loaded dataframe")
train_file.close()

print(df.shape)
print(df)

[30, 60, 90, 120, 150, 180, 210, 240, 270]
Loaded data
Loaded dataframe
(4274, 12)
              coeff1     coeff2    coeff3     coeff4     coeff5     coeff6  \
(0, 0, 0)   1.860936  -0.207383  0.261557  -0.214562  -0.171253  -0.118167   
(0, 0, 1)   1.891651  -0.193249  0.235363  -0.249118  -0.112890  -0.112238   
(0, 0, 2)   1.939205  -0.239664  0.258561  -0.291458  -0.041053  -0.102034   
(0, 0, 3)   1.717517  -0.218572  0.217119  -0.228186  -0.018608  -0.137624   
(0, 0, 4)   1.741191  -0.279891  0.196583  -0.236377  -0.032012  -0.090612   
...              ...        ...       ...        ...        ...        ...   
(8, 29, 4)  1.141889  -0.485815  0.288329  -0.687604   0.301889  -0.203278   
(8, 29, 5)  1.162979  -0.340980  0.290894  -0.553881   0.324858  -0.250906   
(8, 29, 6)  1.224771  -0.371558  0.335834  -0.547974   0.384870  -0.203234   
(8, 29, 7)  1.030606  -0.309280  0.311761  -0.466693   0.453942  -0.260364   
(8, 29, 8)  0.966744  -0.323093  0.305467  -0.419457   0.51

In [5]:
# df = new_dataframe()

In [6]:
ran = [31, 35, 88, 44, 29, 24, 40, 50, 29]
bounds = [i for i in iter.accumulate([31, 35, 88, 44, 29, 24, 40, 50, 29])]
print(bounds)

res = {}

speak_count = 0
utter_count = 0
frame_count = -1

test_file = open("./data/ae.test", "r")
print("Loaded test data")

for line in test_file.readlines():
    if line == "\n":
        utter_count +=1
        frame_count = -1

        if utter_count in bounds:
            speak_count = bounds.index(utter_count) + 1

        res[speak_count] = res.get(speak_count, 0) + 1

    else:
        frame_count += 1

        row = np.asarray(line[:-2].split(" ")).reshape(1, 12)
        # row.append(speak_count)
        # row.append(utter_count)
        # row.append(frame_count)

        utter_idx = utter_count - bounds[speak_count - 1] if speak_count > 0 else utter_count
        # print(utter_idx)
        index = pd.MultiIndex.from_tuples([(int(speak_count), int(utter_idx), int(frame_count))], names=["speak", "utter", "frame"])
        row = pd.DataFrame(row, index=index, columns=colNames)

        #print(row)
        df = pd.concat([df, row])

print("Loaded dataframe")
test_file.close()
for idx in range(len(ran)):
    print(f"{idx}: {ran[idx]}, {res[idx]}")

[31, 66, 154, 198, 227, 251, 291, 341, 370]
Loaded test data
Loaded dataframe
0: 31, 30
1: 35, 35
2: 88, 88
3: 44, 44
4: 29, 29
5: 24, 24
6: 40, 40
7: 50, 50
8: 29, 29


In [7]:
print(df)
print(df.shape)

               coeff1     coeff2     coeff3     coeff4     coeff5     coeff6  \
(0, 0, 0)    1.860936  -0.207383   0.261557  -0.214562  -0.171253  -0.118167   
(0, 0, 1)    1.891651  -0.193249   0.235363  -0.249118  -0.112890  -0.112238   
(0, 0, 2)    1.939205  -0.239664   0.258561  -0.291458  -0.041053  -0.102034   
(0, 0, 3)    1.717517  -0.218572   0.217119  -0.228186  -0.018608  -0.137624   
(0, 0, 4)    1.741191  -0.279891   0.196583  -0.236377  -0.032012  -0.090612   
...               ...        ...        ...        ...        ...        ...   
(8, 28, 6)   1.216506  -0.424432  -0.034349  -0.294238   0.166270  -0.397758   
(8, 28, 7)   1.214579  -0.399925  -0.127891  -0.239583   0.221423  -0.374714   
(8, 28, 8)   1.170031  -0.306025  -0.145534  -0.195854   0.276371  -0.346794   
(8, 28, 9)   1.118108  -0.258605  -0.103982  -0.231256   0.344866  -0.223807   
(8, 28, 10)  1.177449  -0.404080   0.052026  -0.284812   0.429840  -0.203484   

                coeff7     coeff8     c

In [10]:
# Test that all occurences were counted correctly
speakers = dict()
for index, row in df.iterrows():
    speakers[index[0]] = speakers.get(index[0], 0) + 1

type(df)
df.dtypes
for i in range(1, 13):
    df = df.astype({f'coeff{i}': 'float32'})

df.dtypes
print(res)
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

{0: 1096, 1: 991, 2: 1614, 3: 1473, 4: 782, 5: 963, 6: 1170, 7: 1011, 8: 861}


In [ ]:
def splitData(dataset, split_fact):
    df1 = dataset.iloc[:int(split_fact[0]*len(dataset)),:]
    df2 = dataset.iloc[int(split_fact[0]*len(dataset)):,:]
    df3 = df2.iloc[int((split_fact[1]/(split_fact[1] + split_fact[2]))*len(df2)):,:]
    df2 = df2.iloc[:int((split_fact[1]/(split_fact[1] + split_fact[2]))*len(df2)),:]

    # t1 = df1.pop('speaker_count')
    # t2 = df2.pop('speaker_count')
    # t3 = df3.pop('speaker_count')

    # df1 = np.expand_dims(df1, 1)
    # df2 = np.expand_dims(df2, 1)
    # df3 = np.expand_dims(df3, 1)

    # return df1, t1, df2, t2, df3, t3
    return df1, df2, df3
# Convert df to DataSet

In [ ]:
train_x, test_x, val_x = splitData(df, [0.7, 0.2, 0.1])
print(len(train_x), len(test_x), len(val_x))
print((len(train_x) + len(test_x) + len(val_x)), len(df))
print(df)

In [ ]:
with open('Pickles/train_x.txt', 'wb') as tx:
    pickle.dump(train_x, tx)

# with open('Pickles/train_y.txt', 'wb') as ty:
#     pickle.dump(train_y, ty)

with open('Pickles/test_x.txt', 'wb') as tx:
    pickle.dump(test_x, tx)
#
# with open('Pickles/test_y.txt', 'wb') as ty:
#     pickle.dump(test_y, ty)

with open('Pickles/val_x.txt', 'wb') as vx:
    pickle.dump(val_x, vx)
#
# with open('Pickles/val_y.txt', 'wb') as vy:
#     pickle.dump(val_y, vy)
print("Done!")